 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="#Running-W2V" data-toc-modified-id="Running-W2V-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Running W2V</a></span></li></ul></div>

## Data Preparation

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("emails.csv")

In [4]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
def extract_message(string):
    start = string.find('FileName:')
    sub = string[start: ].find('\n')
    return string[start: ][sub:]

In [6]:
df['content'] = df.message.apply(extract_message)

In [7]:
df.content.head(5)

0                        \n\nHere is our forecast\n\n 
1    \n\nTraveling to have a business meeting takes...
2                   \n\ntest successful.  way to go!!!
3    \n\nRandy,\n\n Can you send me a schedule of t...
4              \n\nLet's shoot for Tuesday at 11:45.  
Name: content, dtype: object

In [8]:
import re
def getWords(text):
    return " ".join(re.compile('\w*[A-Za-z]\w*').findall(text))

In [9]:
from nltk.tokenize import TreebankWordTokenizer
def token(text):
    tokenizer = TreebankWordTokenizer()
    return tokenizer.tokenize(text)

In [10]:
df_test = df[1:50]

In [11]:
df_test['words'] = df_test.content.apply(getWords)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_test['words'] = df_test.content.apply(token)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_test.words.head()

1    [Traveling, to, have, a, business, meeting, ta...
2            [test, successful., way, to, go, !, !, !]
3    [Randy, ,, Can, you, send, me, a, schedule, of...
4         [Let, 's, shoot, for, Tuesday, at, 11:45, .]
5    [Greg, ,, How, about, either, next, Tuesday, o...
Name: words, dtype: object

In [14]:
df['words'] = df.content.apply(getWords)

In [15]:
df.words.head()

0                                 Here is our forecast
1    Traveling to have a business meeting takes the...
2                            test successful way to go
3    Randy Can you send me a schedule of the salary...
4                           Let s shoot for Tuesday at
Name: words, dtype: object

In [16]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [17]:
test = df_shuffled[0:1000]

## Running W2V

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [19]:
n_features = 1000

In [20]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(test.words)

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
n_topics = 30
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=42)

In [23]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=30, perp_tol=0.1, random_state=42,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [24]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [25]:
n_top_words = 20

In [26]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
sent message original subject pm october thursday monday tuesday friday wednesday thanks november fw kevin cc john january file august
Topic #1:
deal mike thanks chris need deals cc know kate gas subject new let bob hall position price project day pm
Topic #2:
attached list thanks ena mark report documents leslie xls sheila file questions review time send information summary document comments agreements
Topic #3:
enron corp pm cc subject et na houston meeting forwarded thanks north smith fax office mary john america scott communications
Topic #4:
market commission size new ferc day york end color dynegy face iso net begin october power prices bids order flow
Topic #5:
image updated committee group season credit sunday groups free session game jay ben com members research action agent kevin time
Topic #6:
ees enron na jeff richard ferc iso james steve market california subject cc steffes mara susan shapiro steven kean dasovich
Topic #7:
ect hou enron enron_development corp ees

In [27]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
23     8.968224        1       1  0.010947 -0.106785
7      8.829603        1       2 -0.270587 -0.028141
17     7.440937        1       3  0.177601  0.019113
3      5.757434        1       4 -0.244219 -0.042034
25     4.424053        1       5 -0.235628  0.219664
22     4.072175        1       6  0.089699 -0.058859
21     3.940094        1       7  0.165010  0.174930
6      3.923037        1       8 -0.106913 -0.118569
15     3.716858        1       9  0.025603  0.278390
0      3.715821        1      10 -0.226746  0.018233
11     3.465466        1      11  0.091294  0.031938
16     3.460207        1      12  0.045216 -0.084865
28     3.272317        1      13  0.073318  0.139960
14     3.114630        1      14  0.212063 -0.036663
18     2.944684        1      15 -0.003696 -0.027304
9      2.811089        1      16 -0.077292 -0.130713
29     2.621896        1      17 -0.167842  0.153269
1      2.610972        1      18 -0.060095 -0.128156
4      2.574495        1      19  0.106226 -0.089701
5      2.353001        1      20  0.085141  0.042463
8      2.241845        1      21  0.129837 -0.040070
27     2.098494        1      22  0.135228 -0.056835
20     1.919130        1      23  0.010571 -0.178252
19     1.872802        1      24 -0.153159 -0.015391
2      1.605587        1      25 -0.036560 -0.119150
13     1.559916        1      26  0.083753  0.149773
24     1.555996        1      27  0.102258  0.030177
10     1.315267        1      28 -0.052081  0.045760
12     0.975390        1      29  0.062452  0.017268
26     0.838578        1      30  0.028603 -0.059452, topic_info=     Category         Freq               Term        Total  loglift  logprob
term                                                                        
258   Default  2319.000000                ect  2319.000000  30.0000  30.0000
136   Default  2222.000000                com  2222.000000  29.0000  29.0000
278   Default  3267.000000              enron  3267.000000  28.0000  28.0000
398   Default  1216.000000                hou  1216.000000  27.0000  27.0000
405   Default   489.000000               http   489.000000  26.0000  26.0000
804   Default   457.000000               sent   457.000000  25.0000  25.0000
551   Default   487.000000            message   487.000000  24.0000  24.0000
616   Default   369.000000           original   369.000000  23.0000  23.0000
672   Default   632.000000              power   632.000000  22.0000  22.0000
868   Default   890.000000            subject   890.000000  21.0000  21.0000
262   Default   453.000000                ees   453.000000  20.0000  20.0000
277   Default   559.000000             energy   559.000000  19.0000  19.0000
660   Default   732.000000                 pm   732.000000  18.0000  18.0000
992   Default   367.000000                www   367.000000  17.0000  17.0000
532   Default   392.000000             market   392.000000  16.0000  16.0000
963   Default   178.000000              vince   178.000000  15.0000  15.0000
568   Default   352.000000                 na   352.000000  14.0000  14.0000
113   Default   577.000000                 cc   577.000000  13.0000  13.0000
900   Default   360.000000             thanks   360.000000  12.0000  12.0000
518   Default   348.000000               mail   348.000000  11.0000  11.0000
279   Default   290.000000  enron_development   290.000000  10.0000  10.0000
773   Default   309.000000               said   309.000000   9.0000   9.0000
352   Default   396.000000                gas   396.000000   8.0000   8.0000
544   Default   253.000000            meeting   253.000000   7.0000   7.0000
173   Default   538.000000               corp   538.000000   6.0000   6.0000
56    Default   204.000000           attached   204.000000   5.0000   5.0000
324   Default   118.000000              final   118.000000   4.0000   4.0000
785   Default   121.000000  